In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
import os
import logging
import glob
import json

import numpy as np
import matplotlib.pyplot as plt
# from matplotlib.ticker import StrMethodFormatter
# import matplotlib.gridspec as gridspec
import seaborn as sns
from PIL import Image

import pandas as pd

# from sklearn import metrics
# from sklearn.metrics import confusion_matrix, plot_confusion_matrix, roc_curve, mean_squared_error, mean_absolute_error
import tensorflow as tf

# from zoobot.estimators import make_predictions, bayesian_estimator_funcs
# from zoobot.tfrecord import read_tfrecord
# from zoobot.uncertainty import discrete_coverage
# from zoobot.estimators import input_utils, losses, dirichlet_stats
# from zoobot.tfrecord import catalog_to_tfrecord
# from zoobot.active_learning import metrics, simulated_metrics, acquisition_utils, check_uncertainty, simulation_timeline, run_estimator_config
# from zoobot.active_learning import acquisition_utils
# from zoobot import label_metadata


ModuleNotFoundError: No module named 'zoobot'

In [3]:
questions = label_metadata.decals_questions
label_cols = label_metadata.decals_label_cols
version = 'decals'

schema = losses.Schema(label_cols, questions, version=version)
schema.questions

NameError: name 'label_metadata' is not defined

In [ ]:

# previously shared in draft paper
# predictions_loc = 'results/temp/decals_n2_allq_m0_all.csv'
# eval_loc = 'results/temp/decals_n2_allq_m0_eval_shards/eval_df.csv'

eval_loc = 'results/decals_dr_full_eval_df.csv'
eval_df = pd.read_csv(eval_loc)
eval_df['local_png_loc'] = eval_df['local_png_loc'].apply(lambda x: x.replace('/data/phys-zooniverse/chri5177', '/media/walml/beta/decals'))
# eval_df doesn't include total-votes, add it
for question in schema.questions:
    eval_df[question.text + '_total-votes'] = np.array([eval_df[answer.text].values for answer in question.answers]).sum(axis=0)

# one model
# predictions_loc = 'results/decals_dr_full_m0_eval_predictions.csv'  # .1-20
# predictions = pd.read_csv(predictions_loc)
# samples = dirichlet_stats.load_all_concentrations(eval_predictions, concentration_cols=[a.text + '_concentration' for a in schema.answers])

# several models (ensemble)
samples = []
predictions_locs = [f'results/decals_dr_full_m{n}_eval_predictions.csv' for n in range(3)]  # 0, 1, 2
for predictions_loc in predictions_locs:
    predictions = pd.read_csv(predictions_loc)
    # filter to eval
    eval_predictions = predictions[predictions['iauname'].isin(eval_df['id_str'])]
    print(len(eval_predictions))
    model_samples = dirichlet_stats.load_all_concentrations(eval_predictions, concentration_cols=[a.text + '_concentration' for a in schema.answers])
    samples.append(model_samples)
samples = np.concatenate(samples, axis=2)  # must be same iaunames, sorted

print(samples.shape)



In [ ]:
is_retired = eval_predictions['smooth-or-featured_total-votes'] > 34
retired = eval_predictions[is_retired]
retired['file_loc'] = retired['local_png_loc'].apply(lambda x: '/media/walml/beta/decals' + x.replace('/data/phys-zooniverse/chri5177', ''))
retired_samples = samples[is_retired]


In [ ]:
predicted_fractions = dirichlet_stats.dirichlet_prob_of_answers(retired_samples, schema, temperature=None)